# Beautiful Soup(bs4) を使ってブログの内容をテキストで抜き出してみる
1. リクエスト取得
2. bs4で解析・テキスト取得
3. (改良版) bs4で解析・テキスト取得

In [1]:
# ライブラリインポート
from bs4 import BeautifulSoup
import requests

## リクエスト取得

In [78]:
url = "https://dev.classmethod.jp/cloud/aws/aws-nw-architectures-net320/"
# url = "https://dev.classmethod.jp/cloud/aws/cloudtrail-athena-s3-object-survey/"
response = requests.get(url)
# 確認
response.text[:100]

'\ufeff<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" prefix="og: http://ogp.me/ns#">\n<head>\n<'

## bs4で解析・テキスト取得

In [79]:
soup = BeautifulSoup(response.text, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [80]:
# 単純に get_text()
soup.get_text()[:100]

'\ufeff\n\n\n\n[レポート] AWS ネットワークアーキテクチャ 総まとめ！ #NET320 #reinvent ｜ Developers.IO\n(window.NREUM||(NREUM={})).loa'

In [81]:
# フィルターを行い get_text()
contents = soup.find('div', class_="single_article_contents")
contents.get_text()[:200].split('\n')

['',
 'こちらはラスベガスで開催された AWS re:Invent2019のセッション',
 'The right AWS network architecture for the right reason #NET320',
 'のレポートです。',
 'Transit Gateway/PrivateLink などの新サービス登場や',
 '既存サービスのアップデートとともに、',
 'AWSにおけるネットワーク構成の選択肢は増え続けていま']

## (改良版) bs4で解析・テキスト取得

### ▼ パラグラフのみ抽出

In [82]:
# p 要素の抽出
texts_p = [c.get_text() for c in contents.find_all('p')]
texts_p[:10]

['こちらはラスベガスで開催された AWS re:Invent2019のセッション\nThe right AWS network architecture for the right reason #NET320\nのレポートです。',
 'Transit Gateway/PrivateLink などの新サービス登場や\n既存サービスのアップデートとともに、\nAWSにおけるネットワーク構成の選択肢は増え続けています。',
 '本セッションは、今のAWSにおけるネットワーク構成が網羅されている\n良いセッションでした。',
 '本ブログでは、このセッションで出てきた\nAWSネットワークアーキテクチャ パターンを紹介 していきます。',
 '',
 '項目が多いので以下に目次を作成しています。\n目次のリンクから気になるアーキテクチャを参照ください。',
 '',
 'ネットワークインフラを構築するときに、まず作成するのは VPCです。\nその VPCをどのように構成するかは重要です。',
 '本章は 1 VPCでシステムを構築・運用する構成 について紹介します。',
 '']

In [84]:
import re

# p 要素の抽出
texts_p = [c.get_text() for c in contents.find_all('p')]
# 空白行削除 + 改行コード削除
texts_p = [t.replace('\n','') for t in texts_p if re.match('\S', t)]
texts_p[:10]

['こちらはラスベガスで開催された AWS re:Invent2019のセッションThe right AWS network architecture for the right reason #NET320のレポートです。',
 'Transit Gateway/PrivateLink などの新サービス登場や既存サービスのアップデートとともに、AWSにおけるネットワーク構成の選択肢は増え続けています。',
 '本セッションは、今のAWSにおけるネットワーク構成が網羅されている良いセッションでした。',
 '本ブログでは、このセッションで出てきたAWSネットワークアーキテクチャ パターンを紹介 していきます。',
 '項目が多いので以下に目次を作成しています。目次のリンクから気になるアーキテクチャを参照ください。',
 'ネットワークインフラを構築するときに、まず作成するのは VPCです。その VPCをどのように構成するかは重要です。',
 '本章は 1 VPCでシステムを構築・運用する構成 について紹介します。',
 '一番シンプルな構成です。システム領域の分割は サブネットやルートテーブル、NACLなどを用いて行います。',
 'システムの規模が大きくなって使えるIPレンジが無くなってきた…',
 'その場合は VPCの CIDR拡張 が行えます。']

### ▼ リストアイテムのみ抽出

In [121]:
from bs4.element import Tag, NavigableString

def parse_li(li):
    """
    リストアイテム(li)のテキストを返す
    ※ li内の入れ子リストは除外する ( find_all('li') でそれ単体のリストアイテムが得られるため)
    """
    buffer = []
    for child in li:
        if type(child) == NavigableString:
            buffer.append(child.string)
        elif type(child) == Tag:
            # リスト構造ではない child のみ返り値に含める
            if child.find_all('li') == []:
                buffer.append(child.get_text())
    return ''.join(buffer)

# li 要素の抽出
texts_li = [parse_li(li) for li in contents.find_all('li')]
# 空白行削除 + 改行コード削除
texts_li = [t.replace('\n','') for t in texts_li if re.match('\S', t)]
texts_li[:10]

['フラットネットワーク アーキテクチャ (Single VPC)',
 'シングルアカウント構成',
 'マルチアカウント構成 (Resource Access Manager)',
 '分割ネットワーク アーキテクチャ (Multi VPC)',
 'シングルアカウント構成',
 'マルチアカウント構成',
 'VPC間の接続 (VPC ピアリング)',
 'VPC間の接続 (Transit Gateway)',
 'サイト間VPN',
 'Transit Gateway']

### 改行コードの削除

In [46]:
from bs4.element import Tag, NavigableString

# Child のパース関数
def parse_child(child):
    text = ""
    if type(child) == NavigableString:
        text = child.string
    elif type(child) == Tag:
        text = child.get_text()
    # 改行コードの削除
    return text.replace('\n','')

In [47]:
# 改行コード削除
text_list = [parse_child(c) for c in contents.children]
# 空白行削除
text_list = [s for s in text_list if s]
# 表示
text_list[:15]

['今では S3のセキュリティを高める機能として ブロックパブリックアクセス があります。',
 'S3で誤ったデータの公開を防ぐパブリックアクセス設定機能が追加されました',
 'そのため、これから新規に作成したバケットに対して、誤って パブリックアクセス可能なS3オブジェクト(以降パブリックオブジェクト) を投稿することは未然に防げます。',
 'もちろん既にあるバケットについても、ブロックパブリックアクセスを有効にすることで パブリックオブジェクトの投稿を防げますが、懸念は 本番稼働のアプリケーションへの影響 です。',
 '例えば、あるアプリケーション(SDK)がパブリック読み取りの設定で S3の PutObject を行っていた場合です。ブロックパブリックアクセスを有効にすると、このPutObjectが 403(Access Denied) を返すようになります。',
 'そのため、セキュリティ対策として 「とりあえず、ブロックパブリックアクセスをONに」はリスクがあります。',
 '今回は事前に「特定のS3バケット」に対して パブリックオブジェクトを投稿していないかをCloudTrail と Athena を使って調査する環境を作ってみます。',
 '目次',
 '概要やってみるおわりに',
 '概要',
 'S3バケット(データ格納用)に対して パブリックオブジェクトを投稿しますS3操作(投稿)のログは CloudTrailを介して S3バケット(ログ格納用) に格納されますAthena を使ってパブリックオブジェクト投稿の証拠を確認します',
 'やってみる',
 'データ投稿用のS3バケットは作成済みとして、以降で CloudTrail, Athena を触っていきます。',
 'CloudTrail: 証跡の作成',
 'CloudTrail > 証跡情報 から「証跡の作成」を選択します。']

### リストの各アイテムを 1行ずつ表示

In [48]:
# ul 要素のパース関数
def parse_ul_tag(child):
    if type(child) == NavigableString:
        return [child.string]
    if child.name == 'li':
        # li 要素の中に ul 要素がない場合はそのまま get_text()
        if 'ul' not in [cc.name for cc in child if type(cc) == Tag]:
            return [child.get_text()]
    return sum([parse_ul_tag(cc) for cc in child],[])
   
# Child のパース関数
def parse_child(child):
    texts = []
    # NavigableString の場合、そのまま改行コードを削除して返す
    if type(child) == NavigableString:
        texts = [child.string]
    # Tag の場合
    elif type(child) == Tag:
        # ul 要素の場合 各 li の文をリストで改行コードを削除して返す
        if child.name == 'ul':
            texts = parse_ul_tag(child)
        # get_text()の結果を改行コードを削除して返す
        else:
            texts = [child.get_text()]
    else:
        return []
    return [t.replace('\n','') for t in texts]

In [49]:
# 余計な改行コードの削除 + flatten
text_list = sum([parse_child(c) for c in contents.children], [])
# 空白行削除
text_list = [t for t in text_list if t]
# 表示
text_list

['今では S3のセキュリティを高める機能として ブロックパブリックアクセス があります。',
 'S3で誤ったデータの公開を防ぐパブリックアクセス設定機能が追加されました',
 'そのため、これから新規に作成したバケットに対して、誤って パブリックアクセス可能なS3オブジェクト(以降パブリックオブジェクト) を投稿することは未然に防げます。',
 'もちろん既にあるバケットについても、ブロックパブリックアクセスを有効にすることで パブリックオブジェクトの投稿を防げますが、懸念は 本番稼働のアプリケーションへの影響 です。',
 '例えば、あるアプリケーション(SDK)がパブリック読み取りの設定で S3の PutObject を行っていた場合です。ブロックパブリックアクセスを有効にすると、このPutObjectが 403(Access Denied) を返すようになります。',
 'そのため、セキュリティ対策として 「とりあえず、ブロックパブリックアクセスをONに」はリスクがあります。',
 '今回は事前に「特定のS3バケット」に対して パブリックオブジェクトを投稿していないかをCloudTrail と Athena を使って調査する環境を作ってみます。',
 '目次',
 '概要やってみるおわりに',
 '概要',
 'S3バケット(データ格納用)に対して パブリックオブジェクトを投稿します',
 'S3操作(投稿)のログは CloudTrailを介して S3バケット(ログ格納用) に格納されます',
 'Athena を使ってパブリックオブジェクト投稿の証拠を確認します',
 'やってみる',
 'データ投稿用のS3バケットは作成済みとして、以降で CloudTrail, Athena を触っていきます。',
 'CloudTrail: 証跡の作成',
 'CloudTrail > 証跡情報 から「証跡の作成」を選択します。',
 '今回は特定(東京リージョン)のS3バケットに対象を絞るので、証跡情報を全てのリージョンに適用 はしません。',
 '管理イベント、Insights イベントは取得しません。(※ 各イベントの内容については AWS: CloudTrail イベントとは を参照)',
 '本題の データーイベントの設定 を行います。(+) S3バケット

### その他 (空白削除 + 大文字化)

In [384]:
# 余計な改行コードの削除 + flatten
text_list = sum([parse_child(c) for c in contents.children], [])
# 空白行削除 + 空白削除 + 大文字化
text_list = [t.replace(' ','').upper() for t in text_list if t]
# 表示
text_list

['こちらはラスベガスで開催されたAWSRE:INVENT2019のセッションTHERIGHTAWSNETWORKARCHITECTUREFORTHERIGHTREASON#NET320のレポートです。',
 'TRANSITGATEWAY/PRIVATELINKなどの新サービス登場や既存サービスのアップデートとともに、AWSにおけるネットワーク構成の選択肢は増え続けています。',
 '本セッションは、今のAWSにおけるネットワーク構成が網羅されている良いセッションでした。',
 '本ブログでは、このセッションで出てきたAWSネットワークアーキテクチャパターンを紹介していきます。',
 '資料',
 'セッション動画',
 '目次',
 '項目が多いので以下に目次を作成しています。目次のリンクから気になるアーキテクチャを参照ください。',
 'シングルVPC、マルチVPC',
 'フラットネットワークアーキテクチャ(SINGLEVPC)',
 'シングルアカウント構成',
 'マルチアカウント構成(RESOURCEACCESSMANAGER)',
 '分割ネットワークアーキテクチャ(MULTIVPC)',
 'シングルアカウント構成',
 'マルチアカウント構成',
 'VPC間の接続(VPCピアリング)',
 'VPC間の接続(TRANSITGATEWAY)',
 'ハイブリッドネットワーク',
 'サイト間VPN',
 'TRANSITGATEWAY',
 '仮想プライベートゲートウェイ',
 'ソフトウェアVPNONEC2',
 'DIRECTCONNECT',
 '仮想プライベートゲートウェイ',
 'DIRECTCONNECTGATEWAY+仮想プライベートゲートウェイ',
 'DIRECTCONNECTGATEWAY+TRANSITGATEWAY',
 'IP重複対策',
 'AWS→オンプレミス向き(PRIVATELINK+NLB)',
 'オンプレミス→AWS向き(PRIVATELINK+NLB)',
 'DNS',
 'ROUTE53RESOLVER',
 'サービス別(TRANSITGATEWAY、PRIVATELINK)',
 'TRANSITGATEWAY',
 'SHAREDSERVICEVPC',
 'BUMP-IN-THE-WIREVP

## Ginza を試してみる

In [366]:
# インポート
import pandas as pd
from IPython.display import display
import spacy
pd.set_option('display.max_rows', 300)
nlp = spacy.load('ja_ginza')

In [367]:
# DataFrame 変化用
def doc_to_df(doc):
    cols = ("text", "lemma", "pos", "explain", "stopword")
    rows = [] 
    for t in doc:    
        row = [t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.is_stop]    
        rows.append(row)
    return pd.DataFrame(rows, columns=cols)

In [368]:
doc_blog = nlp(('。'.join(contents_mod)).replace("。。","。"))
df_blog = doc_to_df(doc_blog)
display(df_blog)

,text,lemma,pos,explain,stopword
0,こちら,此方,PRON,pronoun,False
1,は,は,ADP,adposition,True
2,ラスベガス,ラスベガス,PROPN,proper noun,False
3,で,で,ADP,adposition,True
4,開催,開催,VERB,verb,False
...,...,...,...,...,...
2197,得,得る,VERB,verb,False
2198,られ,られる,AUX,auxiliary,True
2199,まし,ます,AUX,auxiliary,False
2200,た,た,AUX,auxiliary,True


In [369]:
# NOUN のみ抽出、ソート
df_blog_noun = df_blog[df_blog["pos"] == "NOUN"]
# 単語の出現回数を数えてみる
df_count = df_blog_noun["text"].value_counts()

display(df_count)

VPC                                                53
TransitGateway                                     21
アーキテクチャ                                            20
ネットワーク                                             20
間                                                  18
構成                                                 18
接続                                                 17
PrivateLink                                        15
アカウント                                              15
App                                                12
TGW                                                11
セッション                                              11
プライベート                                              9
-                                                   9
サービス                                                9
VPN                                                 9
ハイブリッド                                              9
方法                                                  8
IP                          

In [370]:
from spacy import displacy
displacy.render(doc_blog, style="ent")